# Assigment 2 - Analyzing the Data

- Use the data in the **proshares_analysis_data.xlsx** It has monthly data on financial indexes and ETFs from August 2011 through September 2021

1. For the series in the "hedge_fund_series" tab, report the following summary statistics:
   1. mean
   2. volatility
   3. Sharpe Ratio
Annualize these statistics 



In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import statsmodels.api as sm 
import seaborn as sns
import scipy as scs
import sklearn
from sklearn.linear_model import LinearRegression
from scipy import stats 

In [4]:
plt.style.use("seaborn")
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline


In [11]:
# Import Data
file_path = "C:/Users/dcste/OneDrive/Portfolio_Theory/Homework_Jupyter/portfolio_theory/proshares_analysis_data.xlsx"
descriptions = pd.read_excel(file_path, sheet_name = "descriptions")
descriptions.columns = ["Ticker","Security Name"]
descriptions

,Ticker,Security Name
0,EEM US Equity,iShares MSCI Emerging Markets
1,EFA US Equity,iShares MSCI EAFE ETF
2,EUO US Equity,ProShares UltraShort Euro
3,HDG US Equity,ProShares Hedge Replication ET
4,HEFA US Equity,iShares Currency Hedged MSCI E
5,HFRIFWI Index,HFR Fund Wghted Comp
6,IWM US Equity,iShares Russell 2000 ETF
7,MLEIFCTR Index,Merrill Lynch Factor Model Ind
8,MLEIFCTX Index,Merrill Lynch Factor Model Exc
9,QAI US Equity,IndexIQ ETF Trust - IQ Hedge M


In [47]:
hedge_fund = pd.read_excel(file_path, sheet_name = "hedge_fund_series")
hedge_fund = hedge_fund.rename(columns = {"Unnamed: 0" : "Date"}).set_index("Date")
statistics = pd.DataFrame(hedge_fund.mean()*12, columns=["Mean"])
statistics["Volatility"] = hedge_fund.std()*np.sqrt(12)
statistics["Sharpe Ratio"] = statistics["Mean"]/statistics["Volatility"]
statistics.T

,HFRIFWI Index,MLEIFCTR Index,MLEIFCTX Index,HDG US Equity,QAI US Equity
Mean,0.042867,0.025665,0.024278,0.014007,0.011576
Volatility,0.060912,0.056867,0.056685,0.059239,0.048915
Sharpe Ratio,0.703758,0.451308,0.428294,0.236455,0.236649


## Question 2

2. For the Hedge Fund Information, calculate the following statistical related to tail-risk.
   1. Skewness
   2. Excess Kurtosis
   3. VaR(0.05) - the fifth quantile of historic returns 
   4. CVaR(.05) - the mean of the returns at or belwo the fifth quartile
   5. Maximum drawdown - include the dates of the max/min/recovery within the max drawdown period. 

There is no need to annualize any of these statistics

In [88]:
def tail_risk(data,q=0.05):
    df = data.copy()
    df.index = data.index.date
    tail_report = pd.DataFrame(columns = df.columns)
    tail_report.loc['Skewness'] = df.skew()
    tail_report.loc["Excess Kurtosis"] = df.kurtosis()
    tail_report.loc["Value-at-Risk"] = df.quantile(q)
    tail_report.loc["Conditional VaR"] = df[df < df.quantile(q)].mean()
    
    cum_ret = (1+df).cumprod()
    rolling_max = cum_ret.cummax()
    drawdown = (cum_ret - rolling_max)/rolling_max
    tail_report.loc["Max Drawdown"] = drawdown.min()
    tail_report.loc["MDD Start"] = None
    tail_report.loc["MDD End"] = drawdown.idxmin()
    tail_report.loc["Recovery Date"] = None
    
    for col in df.columns:
        tail_report.loc["MDD Start", col] = (rolling_max.loc[:tail_report.loc["MDD End", col]])[col].idxmax()
        recovery_df = (drawdown.loc[tail_report.loc['MDD End',col]:])[col]
        
        try:
            tail_report.loc["Recovery Date", col] = recovery_df[recovery_df >= 0.001].index[0]
            tail_report.loc["Recovery Period (days)"] = (tail_report.loc["Recovery Date"] - tail_report.loc["MDD Start"]).dt.days
        except:
            tail_report.loc["Recovery Date", col] = None
            tail_report.loc["Recovery Period (days)"] = None
    
    
    
    return round(tail_report, 4)

In [89]:
tail_risk(hedge_fund,.05)

C:\Users\dcste\AppData\Local\Temp\ipykernel_7440\2444805183.py:14: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tail_report.loc["MDD Start"] = None
C:\Users\dcste\AppData\Local\Temp\ipykernel_7440\2444805183.py:16: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tail_report.loc["Recovery Date"] = None
C:\Users\dcste\AppData\Local\Temp\ipykernel_7440\2444805183.py:27: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tail_report.loc["Recovery Period (days)"] = None


,HFRIFWI Index,MLEIFCTR Index,MLEIFCTX Index,HDG US Equity,QAI US Equity
Skewness,-1.020683,-0.315513,-0.304807,-0.298573,-0.634129
Excess Kurtosis,6.163102,1.778696,1.741807,1.931106,1.913339
Value-at-Risk,-0.025585,-0.029652,-0.029867,-0.031528,-0.021245
Conditional VaR,-0.039205,-0.036865,-0.036763,-0.038482,-0.034401
Max Drawdown,-0.115473,-0.124302,-0.124388,-0.14072,-0.137714
MDD Start,2019-12-31,2021-06-30,2021-06-30,2021-06-30,2021-06-30
MDD End,2020-03-31,2022-09-30,2022-09-30,2022-09-30,2022-09-30
Recovery Date,None,None,None,None,None
Recovery Period (days),None,None,None,None,None
